<a href="https://colab.research.google.com/github/ThrallPraudmur/Transformers/blob/main/BERT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from collections import Counter
from transformers import GPT2Tokenizer, GPT2Model, BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

marked_data = pd.read_excel('marked_data.xlsx')

encoder = LabelEncoder()

marked_data['Описание'] = marked_data['Описание'].apply(lambda x: str(x))
marked_data['Описание'] = marked_data['Описание'].apply(lambda x: x.lower())

marked_data['Описание'] = marked_data['Описание'].str.replace(r'описание.*', '', regex = True)
marked_data['Описание'] = marked_data['Описание'].str.replace('примечание', '')

#КОДИРОВАНИЕ КЛАССОВ
marked_data['Class'] = encoder.fit_transform(marked_data['Класс'])

X = marked_data['Описание']
y = marked_data['Class']

X_train, X_test, y_train, y_test = train_test_split(marked_data['Описание'], marked_data['Class'], test_size = 0.2, random_state = 42, stratify = marked_data['Class'])

In [ ]:
 #60% - train set, 20% - validation set, 20% - test set
 train, validate, test = np.split(marked_data.sample(frac=1, random_state=42), [int(.6*len(marked_data)), int(.8*len(marked_data))])

 train_text, val_text, test_text = train['Описание'].astype('str'), validate['Описание'].astype('str'), test['Описание'].astype('str')
 train_labels, val_labels, test_labels = train['Class'], validate['Class'], test['Class']

In [ ]:
seq_len = [len(str(i).split()) for i in X_train]
pd.Series(seq_len).hist(bins = 50)

In [ ]:
#compute class frequencies
class_counts = Counter(y_train)
total_samples = len(y_train)

class_weights = torch.tensor([total_samples / (class_counts[i] * len(class_counts)) for i in range(len(class_counts))], dtype = torch.float)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels = 6)

#токенизация и преобразование комментариев
train_encodings = tokenizer(list(X_train), truncation = True, padding = True)
test_encodings = tokenizer(list(X_test), truncation = True, padding = True)

#создание тензоров pytorch для входных данных
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              torch.tensor(y_train.values),
                              class_weights[y_train.values])

test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             torch.tensor(y_test.values))

#создание dataloader для обучающего и тестового набора данных
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle = False)

#установка GPU
device = torch.device('cuda')
model.to(device)

#loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = torch.nn.CrossEntropyLoss(weight = class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)

#обучение модели
for epoch in range(5):
  model.train()
  for batch in train_loader:
    #input_ids, attention_mask, labels = (item.to(device) for item in batch)
    input_ids, attention_mask, labels, weights = (item.to(device) for item in batch)
    optimizer.zero_grad()
    outputs = model(input_ids, attention_mask = attention_mask, labels = labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

model.eval()
predictions = []
with torch.no_grad():
  for batch in test_loader:
    input_ids, attention_mask, _ = (item.to(device) for item in batch)
    outputs = model(input_ids, attention_mask = attention_mask)
    _, predicted_labels = torch.max(outputs.logits, dim = 1)
    predictions.extend(predicted_labels.cpu().numpy())

accuracy = accuracy_score(y_test, predictions)
accuracy

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0.8417910447761194

In [ ]:
y_test = encoder.inverse_transform(y_test)
predictions = encoder.inverse_transform(predictions)

results_df = pd.DataFrame({'Comment': X_test, 'True_Label': y_test, 'Predicted_Label': predictions})
print(classification_report(results_df['True_Label'], results_df['Predicted_Label']))

                            precision    recall  f1-score   support

Возможен повторный контакт       0.81      0.88      0.84        83
                   Интерес       0.94      0.90      0.92        69
      Не интересен продукт       1.00      0.60      0.75        15
        Некачественный лид       0.91      0.88      0.90        73
                     Отказ       0.72      0.82      0.77        57
               Отказ банка       0.77      0.71      0.74        38

                  accuracy                           0.84       335
                 macro avg       0.86      0.80      0.82       335
              weighted avg       0.85      0.84      0.84       335



In [ ]:
pd.options.display.max_colwidth = None
results_df

,Comment,True_Label,Predicted_Label
272,1. : \nне удобно просил перезвонить\n,Возможен повторный контакт,Возможен повторный контакт
621,"1. : \nне удобно говорить, попросили позвонить через час\n",Возможен повторный контакт,Возможен повторный контакт
1042,"1. : \nвыручка — 285,8 млн , недозвон\n2. \nклиенту предварительно одобрен легкий кредит, необходимо отработать на привлечение клиентов на кредитование. информация по клиенту: ооо ""лонопарк"" | адрес: 195279, г. санкт-петербург, проспект индустриальный, д. 45 литера а офис 315 | лпр фио: евстишенков андрей валерьевич - генеральный директор | контактные данные: +7(921)9591905;+7(960)2368563 | регион регистрации: санкт-петербург | вид деятельности, отрасль: деятельность по благоустройству ландшафта | код основного вида деятельности: 81.30 | инн: 7806452340 | окпо: 90839077 | выручка, 2022: 285757000 | чистая прибыль, 2022: 244000 | капитал и резервы, 2022: 1668000 | штат: 9\n",Некачественный лид,Некачественный лид
783,"1. : \nнаталья на связи с менеджером, просила не звонить. сейчас потребности нет, когда будет позвонят менеджеру\n",Возможен повторный контакт,Возможен повторный контакт
999,1. : \nне акт. продавцы\n,Отказ,Некачественный лид
...,...,...,...
382,"1. : \nно, повтор звонка\n",Возможен повторный контакт,Некачественный лид
1444,"1. : \nбк с фд. отказ, нет потребностей\n2. \nпредложить компании инвестиционное кредитование, в рамках пилота ""проектная коробка"". инфо по компании: эстетическая стоматология профессионал, ооо адрес (место нахождения) 119454, г. москва, ул. удальцова, д. 30 руководитель - фио блюмкина ирина михайловна-генеральный директор контакты: +7 (499) 1383215\n+7 (495) 1042404\n+7 (499) 3722361nfo@stomatprofi.ru, stomatprofi@mail.ruww.stomatprofi.ru регион регистрации москва вид деятельности/отрасль стоматологическая практика 2022, выручка, rub 135017000 2022, чистая прибыль (убыток), rub 19233000\n",Отказ,Отказ
97,1. : \nн/о \n,Некачественный лид,Некачественный лид
1495,"1. : \nк гд, работает со сбером и души не чает \n",Отказ,Отказ
